#### AFSK Demodulator
## Step 5: Digital PLL

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_05" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [2]:
from pynq import Overlay, Xlnk
import numpy as np
import pynq.lib.dma

overlay = Overlay('project_05.bit')
dma = overlay.demodulator.dma

## Implementing the Digital PLL

Below is the implementation of the AFSK demodulator in Python.  We are now going to remove the digital PLL code and replace it with new code.  The new code has a slightly different interface.  We get back three bits that we need to unpack into Boolean values.

In [20]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.signal import lfiltic, lfilter, firwin
from scipy.io.wavfile import read
from DigitalPLL import DigitalPLL
from HDLC import HDLC
from AX25 import AX25
import time

block_size = 2640

xlnk = Xlnk()

def demod(data):
    start_time = time.time()
    output = np.array([],dtype=np.bool)
    with xlnk.cma_array(shape=(block_size,), dtype=np.int16) as out_buffer, \
        xlnk.cma_array(shape=(block_size,), dtype=np.int8) as in_buffer:

        for i in range(0, len(data), block_size):
            out_buffer[:len(data[i:i+block_size])] = data[i:i+block_size]
            dma.sendchannel.transfer(out_buffer)
            dma.recvchannel.transfer(in_buffer)
            dma.sendchannel.wait()
            dma.recvchannel.wait()
            output = np.append(output, in_buffer)
            
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('FPGA demodulator execution time: ',sw_exec_time)
    return output

class NRZI:

    def __init__(self):

        self.state = False

    def __call__(self, x):
        
        result = (x == self.state)
        self.state = x
        return result


audio_file = read('../../base/TNC_Test_Ver-1.102-26400-1sec.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us

bpf_delay = 70
lpf_delay = 50
filter_delay = bpf_delay + lpf_delay

# demodulate the audio data
d = demod(audio_data[:26400])

dx = np.append(d, demod(np.zeros(filter_delay)))[filter_delay:]

nrzi = NRZI()
hdlc = HDLC()
for b,s,l in [(bool(x & 4), bool(x & 2), bool(x & 1)) for x in dx]:
    # print(b,s,l)
    if s:
        packet = hdlc(nrzi(b), l)
        if packet is not None:
            print(AX25(packet[1]))


FPGA demodulator execution time:  0.07564449310302734
FPGA demodulator execution time:  0.012141942977905273
KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!?


In [19]:
# xlnk.xlnk_reset()